In [1]:
import torch
import json
import yaml
import pprint
from torch.utils.data import Dataset, DataLoader
from model import HierarchicalInterviewScorer
from dataset import HierarchicalInterviewDataset
from eval import calculate_feedback
from trainer import Trainer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from model_utils import tokenize_dialogue, predict_scores, format_scores


/home/hice1/wyue9/scratch/miniconda3/envs/nl2prot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hice1/wyue9/scratch/miniconda3/envs/nl2prot/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
config_path = "config.yaml"
checkpoint_path = "checkpoint.pth"
test_data_path = "data.json"



In [3]:
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

device = torch.device(config["train"]["device"])

with open(test_data_path, "r") as f:
    test_data = json.load(f)


In [4]:


data_path = "data.json" 
with open(data_path, "r") as f:
    full_data = json.load(f)

if not isinstance(full_data, dict):
    raise ValueError("Expected 'full_data' to be a dictionary.")

keys = list(full_data.keys())
train_keys, val_keys = train_test_split(keys, test_size=0.2, random_state=42)

train_data = {key: full_data[key] for key in train_keys}
val_data = {key: full_data[key] for key in val_keys}

train_data_path = "data/train_data.json"
val_data_path = "data/val_data.json"

with open(train_data_path, "w") as f:
    json.dump(train_data, f, indent=4)
    
with open(val_data_path, "w") as f:
    json.dump(val_data, f, indent=4)

print(f"Training data saved to {train_data_path} with {len(train_data)} samples.")
print(f"Validation data saved to {val_data_path} with {len(val_data)} samples.")


Training data saved to data/train_data.json with 110 samples.
Validation data saved to data/val_data.json with 28 samples.


In [5]:
train_data_path = "data/train_data.json"
val_data_path = "data/val_data.json"

with open(train_data_path, "r") as f:
    train_data = json.load(f)
with open(val_data_path, "r") as f:
    val_data = json.load(f)

train_dataset = HierarchicalInterviewDataset(train_data)
val_dataset = HierarchicalInterviewDataset(val_data)
train_loader = DataLoader(train_dataset, batch_size=config["train"]["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["train"]["batch_size"], shuffle=False)


In [6]:
model = HierarchicalInterviewScorer().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config["train"]["learning_rate"])



trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    max_epochs=config["train"]["max_epochs"]
)


/home/hice1/wyue9/scratch/miniconda3/envs/nl2prot/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
trainer.train()

100%|██████████| 14/14 [00:17<00:00,  1.27s/it]


Epoch 1 | Train Loss: 9.375010405267988
Validation Loss: 1.7431362122297287


100%|██████████| 14/14 [00:10<00:00,  1.33it/s]


Epoch 2 | Train Loss: 1.6420753300189972
Validation Loss: 0.46521060913801193


100%|██████████| 14/14 [00:10<00:00,  1.33it/s]


...

Epoch 98 | Train Loss: 0.28222556731530596
Validation Loss: 0.6127596944570541


100%|██████████| 14/14 [00:10<00:00,  1.32it/s]


Epoch 99 | Train Loss: 0.2713083550333977
Validation Loss: 0.6288924291729927


100%|██████████| 14/14 [00:10<00:00,  1.32it/s]


Epoch 100 | Train Loss: 0.25341132496084484
Validation Loss: 0.8379554823040962


In [8]:
test_dataset = HierarchicalInterviewDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=config["train"]["batch_size"], shuffle=False)

    

In [9]:

all_true_scores = []
all_predicted_scores = []

model.eval()
with torch.no_grad():
    for dialogue_turns, labels in test_loader:
        dialogue_turns = {k: v.to(device) for k, v in dialogue_turns.items()}
        labels = labels.to(device)
        
        predictions = model(dialogue_turns)
        
        all_true_scores.extend(labels.cpu().numpy())
        all_predicted_scores.extend(predictions.cpu().numpy())

all_true_scores_tensor = torch.tensor(all_true_scores)
all_predicted_scores_tensor = torch.tensor(all_predicted_scores)



/scratch/895069/ipykernel_2154089/754747107.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  all_true_scores_tensor = torch.tensor(all_true_scores)


In [10]:

feedback = calculate_feedback(all_predicted_scores_tensor, all_true_scores_tensor)

import pprint
pp = pprint.PrettyPrinter(indent=4)
for idx, candidate_feedback in enumerate(feedback):
    print(f"Feedback for Candidate {idx + 1}:")
    pp.pprint(candidate_feedback)
    print("\n")


Feedback for Candidate 1:
{   'Feedback': 'Acceptable performance in overall score. | Acceptable '
                'performance in recommendation score. | Acceptable performance '
                'in structured answers score.',
    'Overall Score': 5.684344291687012,
    'Recommendation Score': 5.580747604370117,
    'Structured Answers Score': 5.441586017608643}


Feedback for Candidate 2:
{   'Feedback': 'Acceptable performance in overall score. | Acceptable '
                'performance in recommendation score. | Acceptable performance '
                'in structured answers score.',
    'Overall Score': 5.037204265594482,
    'Recommendation Score': 4.888316631317139,
    'Structured Answers Score': 4.301609992980957}


Feedback for Candidate 3:
{   'Feedback': 'Acceptable performance in overall score. | Acceptable '
                'performance in recommendation score. | Acceptable performance '
                'in structured answers score.',
    'Overall Score': 5.2342815399169

In [11]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

candidate_1_data = test_dataset[1]
dialogue_turns, labels = candidate_1_data

print("Original Dialogue for Candidate 1:")
for turn_idx, input_ids in enumerate(dialogue_turns["input_ids"]):
    decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    print(f"Turn {turn_idx + 1}: {decoded_text}")

print(labels)

Original Dialogue for Candidate 1:
Turn 1: so how you doing?
Turn 2: great how about you?
Turn 3: i'm okay.
Turn 4: i'm a little [??? ] by the resurgence of the hurricane but..
Turn 5: i understand i got to get home later too and i'm worried.
Turn 6: yeah we were half way here and it just turned on a switch lightening winds and everything.
Turn 7: oh wow.
Turn 8: so yeah.
Turn 9: so uh tell me about yourself.
Turn 10: okay um i'm a junior here at mit um i'm studying aerospace engineering um my my interests are generally in food mechanics control um design um mostly aircraft but protozoan [??? ] problems in general. um i like to um i like to play music listen to music um i run do some other team sports i like to play hockey.
Turn 11: tell me about a time when you demonstrated leadership.
Turn 12: um okay so uh one of the projects that i've worked on since coming to mit um was during my freshman year i worked in the in the uh artificial intelligence group in uh cesil [ sp ] which is a la

In [14]:
new_dialogue = [
    "Turn 1: so how you doing?"
    "Turn 2: I hate you"
    "Turn 3: i'm okay."
    "Turn 4: ."
    "Turn 5: i understand i got to get home later too and i'm worried."
    "Turn 6: yeah we were half way here and it just turned on a switch lightening winds and everything.",
]

tokenized_input = tokenize_dialogue(new_dialogue)
predicted_scores = predict_scores(model, tokenized_input, device=device)
formatted_scores = format_scores(predicted_scores)
print("Predicted Scores for the New Dialogue:")
for label, score in formatted_scores.items():
    print(f"{label}: {score:.2f}")

Predicted Scores for the New Dialogue:
Overall Score: 5.14
Recommendation Score: 5.13
Structured Answers Score: 6.45


In [15]:
import joblib

In [16]:
model = joblib.load("data/y_overall_model.joblib")
prediction = model.predict("How, are you, I am good.")

/home/hice1/wyue9/scratch/miniconda3/envs/nl2prot/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/hice1/wyue9/scratch/miniconda3/envs/nl2prot/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]